In [ ]:
import os
import math
import pickle
import time
import cv2 as cv
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Ellipse

from config import efficientNet_config
from models import efficientNetV2B0_model

import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
def img_precessing(img, medianblur_kernel, canny_threshold1, canny_threshold2):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                     # 轉成灰階
    img_blur = cv.medianBlur(img_gray, medianblur_kernel)              # median blur 
    img_edges= cv.Canny(img_blur, canny_threshold1, canny_threshold2)  # canny

    return img_gray, img_blur, img_edges

In [ ]:
def Get_canny_angle(img, medianblur_kernel, canny_threshold1, canny_threshold2):
    canny_pred_angle=[]
    img_gray, blur, edges = img_precessing(img, medianblur_kernel, canny_threshold1, canny_threshold2)
    ###############
    cnt, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    '''找全部的輪廓出來畫'''
    allcnt=[]
    for c in cnt:
        allcnt.extend(c.tolist())
    allcnt = np.array(allcnt)

    '''有可能找不到輪廓!!'''
    if len(allcnt) >= 5:  # fit出橢圓至少要有5個cnt
        ''' 找出橢圓 '''
        img_elli = img.copy()
        ellipse  = cv.fitEllipseAMS(allcnt)  # fit出橢圓
        # cv.ellipse(img_elli, ellipse, (0,0,255), 3)
        
        ####### 找出角度(11點鐘方向)
        img_11=img.copy()
        (xc, yc), (width, height), angle = ellipse     # (xc,yc):橢圓中心點 (width, height):橢圓寬長
        # print("最原始angle= ", int(angle))
        
        if angle>90:  # 一開始的angle是比較長短邊的角度
            normal_angle = 360-angle-60
            # print('normal angle=', normal_angle)
            
            ######################################## 處理特殊角度(150~180) ###############################################
            if normal_angle > 150 and normal_angle <=180:   
                normal_angle = normal_angle - 30
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+120)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+120)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+90)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+90)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+30)) * rminor
                # ytop = yc + math.sin(math.radians(angle+30)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+30, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(180~) ################################################
            elif normal_angle > 180:                         
                normal_angle = normal_angle - 60
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+150)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+150)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+120)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+120)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+60)) * rminor
                # ytop = yc + math.sin(math.radians(angle+60)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+60, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+90)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+90)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+60)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+60)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle)) * rminor
                # ytop = yc + math.sin(math.radians(angle)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle, edgecolor='b', fc='None', lw=2, linestyle='dotted')

            # print('new normal angle=', normal_angle)
            # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)
            


        if angle<90:
            normal_angle = 180-angle-60
            # print('normal angle=', normal_angle)
            

            ######################################## 處理特殊角度(0~30) ###############################################  
            if normal_angle > 0 and normal_angle <= 30:   
                normal_angle = normal_angle + 60                            ## !!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+210)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+210)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+180)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+180)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+120)) * rminor
                # ytop = yc + math.sin(math.radians(angle+120)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+120, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 處理特殊角度(30~60) ###############################################
            elif normal_angle > 30 and normal_angle <= 60:   
                normal_angle = normal_angle + 30                            ## !!!!!!!!!!!!!!
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+240)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+240)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+210)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+210)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+150)) * rminor
                # ytop = yc + math.sin(math.radians(angle+150)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+150, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            ######################################## 正常角度(0~150) ################################################
            else:
                canny_pred_angle.append(normal_angle)
                # rmajor = height / 2
                # xtop0  = xc + math.cos(math.radians(angle+270)) * rmajor
                # ytop0  = yc + math.sin(math.radians(angle+270)) * rmajor
                # rmajor = height / 2
                # xtop11 = xc + math.cos(math.radians(angle+240)) * rmajor
                # ytop11 = yc + math.sin(math.radians(angle+240)) * rmajor
                # rminor = width / 2
                # xtop = xc + math.cos(math.radians(angle+180)) * rminor
                # ytop = yc + math.sin(math.radians(angle+180)) * rminor
                # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+180, edgecolor='b', fc='None', lw=2, linestyle='dotted')
            
            # print('new normal angle=', normal_angle)
            # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
            # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
            # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)

        
        ################# plot img
        # plt.figure()
        # f, axarr = plt.subplots(1, 5,figsize=(20,8)) 
        # axarr[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # axarr[1].imshow(cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))
        # axarr[2].imshow(cv.cvtColor(blur, cv.COLOR_BGR2RGB))
        # axarr[3].imshow(cv.cvtColor(edges, cv.COLOR_BGR2RGB))
        # axarr[4].imshow(cv.cvtColor(img_11, cv.COLOR_BGR2RGB))
        # plt.show()
        ##################### save img
        # plt.imsave("../dataset/training_dataset/img/train/Crop_Gray/{}".format(img_name), cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))       # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_MedianBlur/{}".format(img_name), cv.cvtColor(blur, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/training_dataset/img/train/Crop_CannyEdge/{}".format(img_name), cv.cvtColor(edges, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
        # plt.imsave("../dataset/result_img/canny_traindata_direction_result/canny_{}".format(img_name), cv.cvtColor(img_11, cv.COLOR_BGR2RGB))  # !!!!!!!!!!!!

        # plt.figure()
        # f, ax = plt.subplots(figsize=(3,4)) 
        # ax.add_artist(ellipse_info)  # 橢圓
        # ax.set_axis_off()
        # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # plt.savefig("../dataset/result_img/canny_traindata_ellipse_result/ellipse_{}".format(img_name),bbox_inches='tight', pad_inches=0)    # !!!!!!!!!!!!
        # plt.close(f)

    return canny_pred_angle

# 要先判斷是上是下

In [ ]:
model = efficientNetV2B0_model()  # 類別
model.load_weights('../model/202303/20230409.weights')

In [ ]:
# 利用stage 2 bbox切割影像
# 讀影像從stage 1資料夾讀(只需有bbox即可)
# 計算角度

testdata = "test_20220802"

with open('../../yolov4_keras/result_img/{}_stage2_everyframes_result_20230430/{}_stage2_everyframes_result_20230430.pickle'.format(testdata,testdata), 'rb') as f:
    stage2_result = pickle.load(f)


start_time = time.time()
row, col = stage2_result.shape
print("stage2_result shape= ",stage2_result.shape)

image_names       = []
bboxs             = []
canny_pred_angles = []
widths            = []
heights           = []

s_time     = time.time()
loop_times = []
for r in range(row):
    img_name = stage2_result.iloc[r:r+1]['img_name'].values[0]
    x1 = stage2_result.iloc[r:r+1]['x1'].values[0]
    y1 = stage2_result.iloc[r:r+1]['y1'].values[0]
    x2 = stage2_result.iloc[r:r+1]['x2'].values[0]
    y2 = stage2_result.iloc[r:r+1]['y2'].values[0]
    w  = stage2_result.iloc[r:r+1]['w'].values[0]
    h  = stage2_result.iloc[r:r+1]['h'].values[0]
    bbox = (int(x1), int(y1), int(x2), int(y2))
    
    '''crop image by bbox''' 
    img_t = Image.open('../../classification/dataset_test_everyframes/{}/{}'.format(testdata,img_name))
    img   = np.array(img_t.crop(bbox))
    
    '''get angle by canny''' 
    canny_pred_angle = Get_canny_angle(img, medianblur_kernel=9, canny_threshold1=5, canny_threshold2=35)


    '''pred up or down'''
    img_temp = np.array([cv.resize(np.array(img), (224, 224))])
    pred_result = model.predict(img_temp)
    
    
    '''有可能canny偵測不到任何edge，所以直接跳過'''
    if len(canny_pred_angle) != 0:
        image_names.append(img_name)
        bboxs.append(bbox)
        widths.append(w)
        heights.append(h)
        
        if pred_result>0.5:      # >0.5 down
            canny_pred_angles.append(canny_pred_angle[0]+180)
        else:                    # <=0.5 up
            canny_pred_angles.append(canny_pred_angle[0])

        
    else:
        print("canny 沒偵測到= ",img_name)
    
    # 計算每張圖片計算時間
    temp_time = time.time()
    loop_times.append(temp_time-s_time)
    s_time = temp_time

end_time = time.time()

print(end_time-start_time)

In [ ]:
print("Best time on = {:.6f}".format(min(loop_times)))
print("Worst time on = {:.6f}".format(max(loop_times)))
print("Avg time on = {:.6f}".format(sum(loop_times)/len(loop_times)))

In [ ]:
stage3_result = {"img_name":image_names,"bbox":bboxs,"canny_pred_angle":canny_pred_angles,'width':widths,'height':heights}
stage3_result = pd.DataFrame(stage3_result)


In [ ]:
stage3_result.head()

# 在影像上 畫bbox & 畫方向  

In [ ]:
# 創資料夾
row,col = stage3_result.shape

output_folder = '../dataset/result_img/{}_stage3_everyframes_result_20230430/'.format(testdata)
if os.path.isdir(output_folder):
    print("Delete old result folder: {}".format(output_folder))
    os.system("rm -rf {}".format(output_folder))
os.system("mkdir {}".format(output_folder))
print("create folder: {}".format(output_folder))

# 畫在影像左上角

In [ ]:
# for r in range(row):

#    img_name         = stage3_result.iloc[r:r+1]['img_name'].values[0]
#    bbox             = stage3_result.iloc[r:r+1]['bbox'].values[0]
#    canny_pred_angle = stage3_result.iloc[r:r+1]['canny_pred_angle'].values[0]
#    width            = stage3_result.iloc[r:r+1]['width'].values[0]
#    height           = stage3_result.iloc[r:r+1]['height'].values[0]
   
#    # 畫bbox
#    bbox_patches = patches.Rectangle(
#                                  (bbox[0], bbox[1]),   # 起始點
#                                  width, height, edgecolor = 'yellow',linestyle='--', fill=False)
   
#    # 把方向畫在左上角落
#    bbox_xc = (100+100)/2  
#    bbox_yc = (100+100)/2

#    xtop11 = bbox_xc + math.cos(math.radians(canny_pred_angle)) * 100
#    ytop11 = bbox_yc - math.sin(math.radians(canny_pred_angle)) * 100
   
#    uncrop_img = cv.imread('../../classification/dataset_test_everyframes/{}/{}'.format(testdata,img_name))
#    img_rgb    = cv.cvtColor(uncrop_img, cv.COLOR_BGR2RGB)  # 轉成RGB

#    '''plot img'''
#    plt.figure()
#    f, ax = plt.subplots() 
#    ax = plt.gca()
#    ax.add_patch(bbox_patches)
#    plt.plot([0, 200], [100, 100], color="gray", linestyle="--")   # 水平線
#    plt.plot([100, 100], [0, 100], color="gray", linestyle="--")   # 重直線
#    plt.plot([xtop11, bbox_xc], [ytop11, bbox_yc], color="r")      # 11點方向線
#    plt.text(100, 100, int(canny_pred_angle), ha='center', va='top', fontsize=10, color='white')
   
#    ax.set_axis_off()
#    plt.imshow(img_rgb)       # [x1:x2 ,y1:y2]
   
#    plt.savefig(output_folder+"{}".format(img_name),bbox_inches='tight', pad_inches=0)
#    plt.close(f)

# 算平均角度，並製作成字典儲存

In [ ]:
start_imgname = stage3_result.iloc[0:1]['img_name'].values[0]
imgname_front =  start_imgname.split("_")[0]
start_frame_idx = int(start_imgname.split("_")[1].split(".")[0])
print(imgname_front)
end_imgname   = stage3_result.iloc[-1:]['img_name'].values[0]
end_frame_idx = int(end_imgname.split("_")[1].split(".")[0])
print(start_frame_idx, end_frame_idx)


avg_angle_dict = {}  ## 平均角度大表


# 一開始的1秒(30 frames)平均角度都是"NO"
for idx in range(start_frame_idx, start_frame_idx+30):
    imgname   = imgname_front + "_" + str(idx).zfill(6) + ".png"
    avg_angle = "NO"
    avg_angle_dict[imgname] = avg_angle



# 每1秒(30 frames) 取平均
for IDX in range(start_frame_idx, end_frame_idx, 30):  
    canny_pred_angle_in30s = []
    for frame_idx in range(IDX, IDX+30):
        imgname_in30s    = imgname_front + "_" + str(frame_idx).zfill(6) + ".png"
        canny_pred_angle = stage3_result[stage3_result['img_name'] == imgname_in30s]['canny_pred_angle'].values
        canny_pred_angle_in30s.extend(canny_pred_angle)
        
    # 30秒前的平均角度
    if len(canny_pred_angle_in30s)>10:
        avg_angle_in30s = np.array(sorted(canny_pred_angle_in30s)[5:-5]).mean()
    elif len(canny_pred_angle_in30s)==0:
        avg_angle_in30s = 'NO'

    else:
        avg_angle_in30s = np.array(sorted(canny_pred_angle_in30s)).mean()
    
    
    # 從現在開始畫角度，每一秒內的角度都一樣
    for frame_idx in range(IDX+30, IDX+60):
        imgname   = imgname_front + "_" + str(frame_idx).zfill(6) + ".png"
        avg_angle = avg_angle_in30s
        avg_angle_dict[imgname]=avg_angle
    

print(avg_angle_dict)  
    

# 1.畫平均角度在bbox左上角
# 2.畫每一個frame的估計角度在bbox左上角

In [ ]:
for r in range(row): ##row

   img_name         = stage3_result.iloc[r:r+1]['img_name'].values[0]
   bbox             = stage3_result.iloc[r:r+1]['bbox'].values[0]
   canny_pred_angle = stage3_result.iloc[r:r+1]['canny_pred_angle'].values[0]
   width            = stage3_result.iloc[r:r+1]['width'].values[0]
   height           = stage3_result.iloc[r:r+1]['height'].values[0]
   avg_angle        = avg_angle_dict[img_name]

   # 畫bbox
   bbox_patches = patches.Rectangle(
                                 (bbox[0], bbox[1]),   # 起始點
                                 width, height, edgecolor = 'yellow',linestyle='--', fill=False)
   
   # 把方向畫在bbox左上角落
   bbox_xc = (bbox[0]) + (width/4) 
   bbox_yc = (bbox[1]) + (height/4)

   xtop11 = bbox_xc + math.cos(math.radians(canny_pred_angle)) * (width/4)
   ytop11 = bbox_yc - math.sin(math.radians(canny_pred_angle)) * (height/4)

   # 平均角度!="NO"，代表有數值所以顯示
   if avg_angle != "NO":
      xtop11_avg = bbox_xc + math.cos(math.radians(avg_angle)) * (width/4)
      ytop11_avg = bbox_yc - math.sin(math.radians(avg_angle)) * (height/4)
      
      uncrop_img = cv.imread('../../classification/dataset_test_everyframes/{}/{}'.format(testdata,img_name))
      img_rgb    = cv.cvtColor(uncrop_img, cv.COLOR_BGR2RGB)  # 轉成RGB
      
      '''plot img'''
      plt.figure()
      f, ax = plt.subplots() 
      ax = plt.gca()
      ax.add_patch(bbox_patches)
      plt.plot([bbox_xc-(width/4), bbox_xc+(width/4)], [bbox_yc, bbox_yc], color="gray", linestyle="--", lw=0.5)   # 水平線
      plt.plot([bbox_xc, bbox_xc], [bbox_yc-(height/4), bbox_yc], color="gray", linestyle="--", lw=0.5)            # 重直線
      plt.plot([xtop11, bbox_xc], [ytop11, bbox_yc], color="r", linestyle="--", lw=0.5)                            # 11點方向線
      plt.plot([xtop11_avg, bbox_xc], [ytop11_avg, bbox_yc], color="b")                                    # avg angle 的 11點方向線
      # plt.text(bbox_xc, bbox_yc, int(canny_pred_angle), ha='center', va='top', fontsize=10, color='white')  # 標示角度
      
      ax.set_axis_off()
      plt.imshow(img_rgb)       # [x1:x2 ,y1:y2]
      # plt.show()
      plt.savefig(output_folder+"{}".format(img_name),bbox_inches='tight', pad_inches=0)
      plt.close(f)
   

   # 平均角度=="NO"，代表沒有數值所以不顯示
   else:
      uncrop_img = cv.imread('../../classification/dataset_test_everyframes/{}/{}'.format(testdata,img_name))
      img_rgb    = cv.cvtColor(uncrop_img, cv.COLOR_BGR2RGB)  # 轉成RGB
      
      '''plot img'''
      plt.figure()
      f, ax = plt.subplots() 
      ax = plt.gca()
      ax.add_patch(bbox_patches)
      plt.plot([bbox_xc-(width/4), bbox_xc+(width/4)], [bbox_yc, bbox_yc], color="gray", linestyle="--", lw=0.5)   # 水平線
      plt.plot([bbox_xc, bbox_xc], [bbox_yc-(height/4), bbox_yc], color="gray", linestyle="--", lw=0.5)            # 重直線
      plt.plot([xtop11, bbox_xc], [ytop11, bbox_yc], color="r", linestyle="--", lw=0.5)                            # 11點方向線
      # plt.text(bbox_xc, bbox_yc, int(canny_pred_angle), ha='center', va='top', fontsize=10, color='white')  # 標示角度
      
      ax.set_axis_off()
      plt.imshow(img_rgb)       # [x1:x2 ,y1:y2]
      # plt.show()
      plt.savefig(output_folder+"{}".format(img_name),bbox_inches='tight', pad_inches=0)
      plt.close(f)

# 只有畫每一個frame角度在bbox左上角

In [ ]:
# for r in range(row): ##row

#    img_name         = stage3_result.iloc[r:r+1]['img_name'].values[0]
#    bbox             = stage3_result.iloc[r:r+1]['bbox'].values[0]
#    canny_pred_angle = stage3_result.iloc[r:r+1]['canny_pred_angle'].values[0]
#    width            = stage3_result.iloc[r:r+1]['width'].values[0]
#    height           = stage3_result.iloc[r:r+1]['height'].values[0]
#    # print(width,height)
#    # 畫bbox
#    bbox_patches = patches.Rectangle(
#                                  (bbox[0], bbox[1]),   # 起始點
#                                  width, height, edgecolor = 'yellow',linestyle='--', fill=False)
   
#    # 把方向畫在bbox左上角落
#    bbox_xc = (bbox[0]) + (width/4) 
#    bbox_yc = (bbox[1]) + (height/4)

#    xtop11 = bbox_xc + math.cos(math.radians(canny_pred_angle)) * (width/4)
#    ytop11 = bbox_yc - math.sin(math.radians(canny_pred_angle)) * (height/4)
   
#    uncrop_img = cv.imread('../../classification/dataset_test_everyframes/{}/{}'.format(testdata,img_name))
#    img_rgb    = cv.cvtColor(uncrop_img, cv.COLOR_BGR2RGB)  # 轉成RGB

#    '''plot img'''
#    plt.figure()
#    f, ax = plt.subplots() 
#    ax = plt.gca()
#    ax.add_patch(bbox_patches)
#    plt.plot([bbox_xc-(width/4), bbox_xc+(width/4)], [bbox_yc, bbox_yc], color="gray", linestyle="--")   # 水平線
#    plt.plot([bbox_xc, bbox_xc], [bbox_yc-(height/4), bbox_yc], color="gray", linestyle="--")            # 重直線
#    plt.plot([xtop11, bbox_xc], [ytop11, bbox_yc], color="r")                                            # 11點方向線
#    # plt.text(bbox_xc, bbox_yc, int(canny_pred_angle), ha='center', va='top', fontsize=10, color='white')  # 標示角度
   
#    ax.set_axis_off()
#    plt.imshow(img_rgb)       # [x1:x2 ,y1:y2]
#    # plt.show()
#    # plt.savefig(output_folder+"{}".format(img_name),bbox_inches='tight', pad_inches=0)
#    # plt.close(f)